<a href="https://colab.research.google.com/github/JustynaAnd/DataWorkshop-Matrix3/blob/master/hyperparam_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout 
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
!pip install hyperopt

In [25]:
cd '/content/drive/My Drive/Colab Notebooks/DataWorkshop-Matrix/matrix3/DataWorkshop-Matrix3'

/content/drive/My Drive/Colab Notebooks/DataWorkshop-Matrix/matrix3/DataWorkshop-Matrix3


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)

  model.fit(X_train, 
            y_train,
            batch_size = params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_train, y_train)),
            callbacks = [tensorboard_callback]
            )
  return model


def predict(model_trained, X_test, y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),
                    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),
                    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
                    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
                    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Flatten(),
                     
                    Dense(1024, activation = 'relu'),
                    Dropout(0.3),
                    Dense(1024, activation = 'relu'),
                    Dropout(0.3),
                     
                    Dense(num_classes, activation = 'softmax'),
                     ])

In [30]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 9s 35ms/step - loss: 2.3990 - accuracy: 0.3339 - val_loss: 0.7505 - val_accuracy: 0.7856
Epoch 2/5
272/272 [==============================] - 6s 20ms/step - loss: 0.6188 - accuracy: 0.8114 - val_loss: 0.1447 - val_accuracy: 0.9542
Epoch 3/5
272/272 [==============================] - 6s 21ms/step - loss: 0.2986 - accuracy: 0.9087 - val_loss: 0.0811 - val_accuracy: 0.9757
Epoch 4/5
272/272 [==============================] - 5s 20ms/step - loss: 0.2030 - accuracy: 0.9393 - val_loss: 0.0529 - val_accuracy: 0.9840
Epoch 5/5
272/272 [==============================] - 5s 20ms/step - loss: 0.1644 - accuracy: 0.9514 - val_loss: 0.0386 - val_accuracy: 0.9891


0.962358276643991

In [31]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 4ms/step - loss: 0.1574 - accuracy: 0.9624


0.962358295917511

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),
                    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),
                    
                    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
                    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),
                     
                    Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
                    Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),
                     
                    Flatten(),
                     
                    Dense(1024, activation = 'relu'),
                    Dropout(params['dropout_dense_block_one']),
                    Dense(1024, activation = 'relu'),
                    Dropout(params['dropout_dense_block_two']),
                     
                    Dense(num_classes, activation = 'softmax'),
                    ])

In [0]:
def func_obj(params):
  
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy = {}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [36]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),

}


best = fmin(
    func_obj, 
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4388535072361429, 'dropout_cnn_block_three': 0.49282115369549284, 'dropout_cnn_block_two': 0.38926927101802095, 'dropout_dense_block_one': 0.5131208281994434, 'dropout_dense_block_two': 0.5707143284321602}
accuracy = 0.8884353637695312
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.3692637615988131, 'dropout_cnn_block_three': 0.33413273491164336, 'dropout_cnn_block_two': 0.3549282587062857, 'dropout_dense_block_one': 0.6251227378411643, 'dropout_dense_block_two': 0.3381051666728688}
accuracy = 0.9517006874084473
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.40037610708857785, 'dropout_cnn_block_three': 0.3529074425332909, 'dropout_cnn_block_two': 0.3959036747205463, 'dropout_dense_block_one': 0.549131177355896, 'dropout_dense_block_two': 0.31134959337058526}
accuracy = 0.9337868690490723
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4122184644669461, 'dropout_cnn_block_three': 0.4102659218673299, 'dropout_cnn_block_two': 0.456240